# Modelo de Ia (Gaussian)

In [96]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import  train_test_split,KFold,cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.ensemble import RandomForestClassifier

In [97]:
df = pd.read_excel('../base/incluses_tratado.xlsx')

display(df.head(5))

,Participante da Comunidade LGBTQIA+,Faixa Etária,Identidade de Gênero,Orientação Sexual,Cidade/Estado,Escolaridade,Usa Apps para Oportunidades de Emprego?,Preferência de Cursos,Desafios de Emprego por Gênero,Interesse em Empreender,Situação no Mercado de Trabalho,Usa Redes Sociais?,Finalidade do Uso do App Incluses
0,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Sim,Prefiro cursos on-line,Sim,Não,Empregado (a),Sim,Todos acima
1,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Tenho interesse,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,Não usaria o app
2,Sim,Menos de 18 anos,Mulher cisgênero,Pansexual,Região Norte,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Não,Talvez,Empregado (a),Sim,Fazer cursos de qualificação
3,Sim,Menos de 18 anos,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,Não usaria o app
4,Sim,Menos de 18 anos,Homem transgênero,Prefiro não informar,Região Sudeste,Nenhum,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Sim,Desempregado (a),Sim,Fazer cursos de qualificação


In [98]:
df_resposta = df['Finalidade do Uso do App Incluses']
df_atributo = df.iloc[:, :-1]

#Resposta
label_encoder = LabelEncoder()

#Treino
preprocessador = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool'])),
    ],
    remainder='passthrough',
    verbose_feature_names_out= False 
)

In [99]:
df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())
df_resposta = pd.DataFrame(label_encoder.fit_transform(df_resposta))


smote = SMOTE(random_state=42,k_neighbors=2)
df_atributo, df_resposta = smote.fit_resample(df_atributo, df_resposta)

In [100]:
df_atributos_treino, df_atributos_teste, df_resposta_treino, df_resposta_teste = train_test_split(df_atributo, df_resposta, test_size=0.25, random_state=12)

atributos_treino = df_atributos_treino.values
resposta_treino = df_resposta_treino.values

atributos_teste = df_atributos_teste.values
resposta_teste = df_resposta_teste.values




# Aplicar SMOTE no conjunto de treino, ajustando o número de vizinhos para evitar o erro
# smote = SMOTE(random_state=12, k_neighbors=1)
# atributos_treino, resposta_treino = smote.fit_resample(df_atributos_treino, df_resposta_treino)

# Convertendo para arrays
atributos_teste = df_atributos_teste.values
resposta_teste = df_resposta_teste.values

In [101]:
classificador_bayes = GaussianNB()
modelo = classificador_bayes.fit(df_atributos_treino, df_resposta_treino)

probabilidades = classificador_bayes.predict_proba(atributos_teste)
classificacao = classificador_bayes.predict(atributos_teste)

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [102]:
# print("Probabilidades das classes para os dados de teste:\n", probabilidades)
# print("Classificação dos dados de teste:\n", classificacao)

taxa_de_acerto = accuracy_score(resposta_teste, classificacao)
print("Taxa de acerto", taxa_de_acerto)
print("\nRelatório de Classificação:\n", classification_report(resposta_teste, classificacao))
print("Matriz de Confusão:\n", confusion_matrix(resposta_teste, classificacao))

Taxa de acerto 0.6

Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       0.00      0.00      0.00         3
           2       0.80      0.67      0.73         6
           3       0.67      1.00      0.80         2
           4       0.38      0.50      0.43         6

    accuracy                           0.60        20
   macro avg       0.57      0.63      0.59        20
weighted avg       0.57      0.60      0.58        20

Matriz de Confusão:
 [[3 0 0 0 0]
 [0 0 0 0 3]
 [0 0 4 0 2]
 [0 0 0 2 0]
 [0 1 1 1 3]]


In [103]:
numfolds = 5
gnb = GaussianNB()

kf = KFold(n_splits=numfolds, shuffle=True, random_state=31)

validaçãoCruzada = cross_val_score(gnb, df_atributo, df_resposta, cv=kf)
media = validaçãoCruzada.mean()

print(validaçãoCruzada)
print(media)

[0.4375 0.6875 0.6875 0.625  0.5625]
0.6


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-pack

In [104]:
predictions = cross_val_predict(gnb, df_atributo, df_resposta, cv=kf)

fold_indices = np.arange(len(predictions)) % numfolds

conf_matrices = [
    confusion_matrix(df_resposta[fold_indices == i], predictions[fold_indices == i])
    for i in range(numfolds)
]

acuracia_matrices = [
    accuracy_score(df_resposta[fold_indices == i], predictions[fold_indices == i])
    for i in range(numfolds)
]

for i, conf_matrix in enumerate(conf_matrices):
    print(f"Fold {i + 1}:\nAcurácia: {round(acuracia_matrices[i]*100)}%\n{conf_matrix}\n")

mean_conf_matrix = np.mean(conf_matrices, axis=0)
mean_acuracia_matrix = np.mean(acuracia_matrices, axis=0)
print("matriz de confusao média:")
print(f"{mean_acuracia_matrix*100:.2f}%")
print(mean_conf_matrix)

Fold 1:
Acurácia: 69%
[[3 0 0 0 0]
 [1 3 0 0 0]
 [1 1 1 0 0]
 [0 0 0 2 0]
 [1 0 1 0 2]]

Fold 2:
Acurácia: 50%
[[3 0 0 0 0]
 [1 1 2 0 0]
 [0 1 2 0 0]
 [0 2 0 2 0]
 [0 1 1 0 0]]

Fold 3:
Acurácia: 62%
[[3 0 0 0 0]
 [0 3 1 0 0]
 [0 0 4 0 0]
 [0 1 1 0 0]
 [0 2 1 0 0]]

Fold 4:
Acurácia: 69%
[[4 0 0 0 0]
 [0 1 0 0 1]
 [0 0 3 0 0]
 [0 2 1 3 0]
 [0 1 0 0 0]]

Fold 5:
Acurácia: 50%
[[3 0 0 0 0]
 [0 1 1 0 0]
 [0 0 3 0 0]
 [0 0 0 1 1]
 [0 2 3 1 0]]

matriz de confusao média:
60.00%
[[3.2 0.  0.  0.  0. ]
 [0.4 1.8 0.8 0.  0.2]
 [0.2 0.4 2.6 0.  0. ]
 [0.  1.  0.4 1.6 0.2]
 [0.2 1.2 1.2 0.2 0.4]]


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-pack